# First, let us set up a few dependencies

Don't forget to switch to a GPU-enabled colab runtime!

```
Runtime -> Change Runtime Type -> GPU
```

In [1]:
import os
import contextlib
@contextlib.contextmanager
def directory(name):
  ret = os.getcwd()
  print(ret)
  os.chdir(name)
  yield None
  os.chdir(ret)

import subprocess
def run(input, exception_on_failure=False):
  try:
    program_output = subprocess.check_output(f"{input}", shell=True, universal_newlines=True, stderr=subprocess.STDOUT)
  except Exception as e:
    if exception_on_failure:
      raise e
    program_output = e.output

    return program_output
def prun(input, exception_on_failure=False):
  x = run(input, exception_on_failure)
  print(x)
  return x

In [2]:
import os

files = os.listdir(os.getcwd())
for file in files:
    print(file)
print(os.getcwd())


.ipynb_checkpoints
Dockerfile
.dtproject
README.md
.gitignore
packages
docs
.vscode
assets
notebooks
tests
dt_object_detection_training.ipynb
/app


# This mounts your google drive to this notebook. You might have to change the path to fit with your dataset folder inside your drive.

Read the instruction output by the cell bellow carefully!

In [3]:
# Create a temporary workspace
# import tempfile
import os

print(os.getcwd())
# SESSION_WORKSPACE = tempfile.mkdtemp()
SESSION_WORKSPACE = os.path.join(os.getcwd(),"home/dharani/Desktop/VSC_Projects/duckietown-lx/object-detection/assets/")
print(f"Session workspace created at: {SESSION_WORKSPACE}")


/app
Session workspace created at: /app/home/dharani/Desktop/VSC_Projects/duckietown-lx/object-detection/assets/


In [4]:
# # Mount the drive
# from google.colab import drive
# drive.mount('/content/drive')
# DRIVE_PATH = "/content/drive/My Drive"
DRIVE_PATH = SESSION_WORKSPACE + "duckietown_object_detection_dataset"
print(DRIVE_PATH)

/app/home/dharani/Desktop/VSC_Projects/duckietown-lx/object-detection/assets/duckietown_object_detection_dataset


In [ ]:
# Unzip the dataset
import shutil
import os


DATASET_DIR_NAME = "duckietown_object_detection_dataset"
DATASET_ZIP_NAME = f"{DATASET_DIR_NAME}.zip"
DATASET_DIR_PATH = os.path.join(SESSION_WORKSPACE, DATASET_DIR_NAME)
TRAIN_DIR = "train"
VALIDATION_DIR = "val"
IMAGES_DIR = "images"
LABELS_DIR = "labels"


def show_info(base_path: str):
  for l1 in [TRAIN_DIR, VALIDATION_DIR]:
    for l2 in [IMAGES_DIR, LABELS_DIR]:
      p = os.path.join(base_path, l1, l2)
      print(f"#Files in {l1}/{l2}: {len(os.listdir(p))}")


def unzip_dataset():
  # check zipped file
  zip_path = os.path.join(DRIVE_PATH, DATASET_ZIP_NAME)
  assert os.path.exists(zip_path), f"No zipped dataset found at {zip_path}! Abort!"

  # unzip the data
  print("Unpacking zipped data...")
  shutil.unpack_archive(zip_path, DATASET_DIR_PATH)
  print(f"Zipped dataset unpacked to {DATASET_DIR_PATH}")

  # show some info
  show_info(DATASET_DIR_PATH)


unzip_dataset()

In [5]:
# change  working directory to the session workspace
print(os.getcwd())
working_directory = os.path.join(os.getcwd(),"assets")
print(working_directory)
os.chdir(working_directory)
print(f"PWD: {os.getcwd()}")

# install pytorch and torchvision (these versions must match those on the physical duckiebot)
!pip3 install torch==1.7.0 torchvision==0.8.1

/app
/app/assets
PWD: /app/assets

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


# Next, we will clone Yolov5

In [6]:
!rm -rf ./yolov5
!git clone https://github.com/ultralytics/yolov5.git -b v7.0
!git pull
!cd yolov5 && pip3 install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15994, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15994 (delta 18), reused 18 (delta 12), pack-reused 15967
Receiving objects: 100% (15994/15994), 14.65 MiB | 7.32 MiB/s, done.
Resolving deltas: 100% (10984/10984), done.
Note: switching to '915bbf294bb74c859f0b41f1c23bc395014ea679'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

fatal: not a git repository (or any parent up to mount point /)
Stopping 

# We now inform the training process of the format and location of our dataset

In [8]:
%%writefile ./yolov5/data/duckietown.yaml

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../duckietown_object_detection_dataset/train
val: ../duckietown_object_detection_dataset/val

# number of classes
nc: 4

# class names
names: [ 'duckie', 'cone', 'truck', 'bus' ]

Overwriting ./yolov5/data/duckietown.yaml


Found existing installation: Pillow 10.0.1
Uninstalling Pillow-10.0.1:
  Would remove:
    /usr/local/lib/python3.8/site-packages/PIL/*
    /usr/local/lib/python3.8/site-packages/Pillow-10.0.1.dist-info/*
    /usr/local/lib/python3.8/site-packages/Pillow.libs/libXau-154567c4.so.6.0.0
    /usr/local/lib/python3.8/site-packages/Pillow.libs/libbrotlicommon-3ecfe81c.so.1
    /usr/local/lib/python3.8/site-packages/Pillow.libs/libbrotlidec-922c819b.so.1
    /usr/local/lib/python3.8/site-packages/Pillow.libs/libfreetype-82733d78.so.6.20.1
    /usr/local/lib/python3.8/site-packages/Pillow.libs/libharfbuzz-a8a034ff.so.0.60820.0
    /usr/local/lib/python3.8/site-packages/Pillow.libs/libjpeg-58afefff.so.62.4.0
    /usr/local/lib/python3.8/site-packages/Pillow.libs/liblcms2-0821774a.so.2.0.15
    /usr/local/lib/python3.8/site-packages/Pillow.libs/liblzma-1e44b93d.so.5.4.4
    /usr/local/lib/python3.8/site-packages/Pillow.libs/libopenjp2-20e347f0.so.2.5.0
    /usr/local/lib/python3.8/site-packages/

# And we're ready to train! This step will take about 5 minutes.

Notice that we're only training for 10 epochs. That's probably not enough!

In [9]:
!cd yolov5 && python3 train.py --cfg ./models/yolov5n.yaml --img 416 --batch 32 --epochs 10 --data duckietown.yaml --weights yolov5n.pt

train: weights=yolov5n.pt, cfg=./models/yolov5n.yaml, data=duckietown.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 224 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v7.0-0-g915bbf2 Python-3.8.18 torch-1.7.0 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_

In [24]:
import numpy as np

all_exps = os.listdir("yolov5/runs/train")
all_exps_filtered = map(lambda x: int(x.replace("exp", "1")), filter(lambda x: x.startswith("exp"), all_exps))
all_exps_filtered = np.array(list(all_exps))
latest_exp_index = np.argmax(all_exps)
latest_exp = all_exps[latest_exp_index]
print(f"Latest exp is {latest_exp}")

prun(f"cp yolov5/runs/train/{latest_exp}/weights/best.pt yolov5/best.pt")
print(f"Marked the model from the latest run ({latest_exp}) as yolov5/best.pt.")

Latest exp is exp
None
Marked the model from the latest run (exp) as yolov5/best.pt.


In [22]:
import os
import numpy as np

all_exps = os.listdir("yolov5/runs/train")
all_exps_filtered = list(map(lambda x: int(x.replace("exp", "1")), filter(lambda x: x.startswith("exp"), all_exps)))

# If there are no matching directories, exit
if not all_exps_filtered:
    print("No matching directories found!")
    exit()

latest_exp_index = np.argmax(all_exps_filtered)
latest_exp = f"exp{all_exps_filtered[latest_exp_index]}"
print(f"Latest exp is {latest_exp}")

os.system(f"cp yolov5/runs/train/{latest_exp}/weights/best.pt yolov5/best.pt")
print(f"Marked the model from the latest run ({latest_exp}) as yolov5/best.pt.")


Latest exp is exp1
Marked the model from the latest run (exp1) as yolov5/best.pt.


cp: cannot stat 'yolov5/runs/train/exp1/weights/best.pt': No such file or directory


# Next, we can upload your model to Duckietown's cloud!

We will need our token to access our personal cloud space.

In [25]:
# TODO: Fill in the duckietown token here
YOUR_DT_TOKEN = "dt1-3nT7FDbT7NLPrXykNJW6pALtojDqDpApvrDdn7u7qVrhwvL-43dzqWFnWd8KBa1yev1g3UKnzVxZkkTbfakGpMump6ZThdzwSdStn7yTvASbsDof6d"

In [33]:
import os
os.getcwd()
os.path.join(os.getcwd(),"yolov5/best.pt")

'/app/assets'

Then, we chose the location of the trained model on disk and its name once uploaded to our cloud space. You should not change these values, or the robots will not be able to find the model to download.

In [29]:
import sys
sys.path.insert(0, './yolov5')
print(os.getcwd())
# DO NOT CHANGE THESE
model_name = "yolov5n"
model_local_path = "./yolov5/best.pt"
model_remote_path = f"courses/mooc/objdet/data/nn_models/{model_name}.pt"
print(model_local_path)
print(model_remote_path)
# install DCSS client
!pip3 install dt-data-api-daffy

/app/assets
./yolov5/best.pt
courses/mooc/objdet/data/nn_models/yolov5n.pt

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


We now open a pointer to our cloud space and upload the model.

In [30]:
import torch
from dt_data_api import DataClient, Storage

# open a pointer to our personal duckietown cloud space
client = DataClient(YOUR_DT_TOKEN)
storage = client.storage("user")

# upload model
upload = storage.upload(model_local_path, model_remote_path)
upload.join()

# Done!

We're done training! You can now close this tab and go back to the `Training` notebook